### Model Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
DATA_DIR = '/content/drive/MyDrive/EarVN1.0 dataset/Images'
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64

In [4]:
train_val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="training",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 24151 files for training.


In [5]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="validation",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 4261 files for validation.


In [6]:
train_val_batches = tf.data.experimental.cardinality(train_val_ds)
train_val_batches

<tf.Tensor: shape=(), dtype=int64, numpy=378>

In [7]:
TOTAL_TRAIN_VAL = len(train_val_ds)
print(TOTAL_TRAIN_VAL)
train_ds = train_val_ds.take(int(8 * train_val_batches / 10))
val_ds = train_val_ds.skip(int(8 * train_val_batches / 10))
print(len(train_ds))
print(len(val_ds))

378
302
76


In [8]:
train_ds

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
base_model_EfficientNetV2S = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

82420632/82420632 [==============================] - 0s 0us/step


In [10]:
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D

model = tf.keras.Sequential()
model.add(base_model_EfficientNetV2S)
model.add(GlobalMaxPooling2D())
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(164, activation = 'softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 7, 7, 1280)        20331360  
 al)                                                             
                                                                 
 global_max_pooling2d (Glob  (None, 1280)              0         
 alMaxPooling2D)                                                 
                                                                 
 dense (Dense)               (None, 4096)              5246976   
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 164)               671908    
                                                                 
Total params: 26250244 (100.14 MB)
Trainable params: 260

In [12]:
from tensorflow.keras import optimizers, losses
model.compile(optimizer=optimizers.Adam(learning_rate=3e-4), loss='sparse_categorical_crossentropy', metrics='accuracy')

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(
    filepath='./best_efficientnetv2s_model_epoch={epoch:02d}_acc={val_accuracy:0.4f}.h5',
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    save_freq='epoch'
)

In [ ]:
results = model.fit(train_ds, validation_data=val_ds, epochs=50, batch_size=64, callbacks=[mc])

Epoch 1/50
302/302 [==============================] - ETA: 0s - loss: 4.3141 - accuracy: 0.0751
Epoch 1: val_accuracy improved from -inf to 0.30168, saving model to ./best_efficcientnetv2s_model_epoch=01_acc=0.3017.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


302/302 [==============================] - 3234s 10s/step - loss: 4.3141 - accuracy: 0.0751 - val_loss: 2.7444 - val_accuracy: 0.3017
Epoch 2/50
302/302 [==============================] - ETA: 0s - loss: 1.7847 - accuracy: 0.5246
Epoch 2: val_accuracy improved from 0.30168 to 0.72299, saving model to ./best_efficcientnetv2s_model_epoch=02_acc=0.7230.h5
302/302 [==============================] - 383s 1s/step - loss: 1.7847 - accuracy: 0.5246 - val_loss: 1.0135 - val_accuracy: 0.7230
Epoch 3/50
302/302 [==============================] - ETA: 0s - loss: 0.6407 - accuracy: 0.8157
Epoch 3: val_accuracy improved from 0.72299 to 0.80365, saving model to ./best_efficcientnetv2s_model_epoch=03_acc=0.8036.h5
302/302 [==============================] - 382s 1s/step - loss: 0.6407 - accuracy: 0.8157 - val_loss: 0.7220 - val_accuracy: 0.8036
Epoch 4/50
302/302 [==============================] - ETA: 0s - loss: 0.3089 - accuracy: 0.9064
Epoch 4: val_accuracy improved from 0.80365 to 0.84014, saving m